In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import json
from typing import List, Dict
import json
import redis
import time
from uuid import uuid4
from src import settings

In [15]:
# Connect to Redis and assign to variable `db``
# Make use of settings.py module to get Redis settings like host, port, etc.
db = redis.Redis(settings.REDIS_IP, settings.REDIS_PORT, db=settings.REDIS_DB_ID)
db.ping()

True

In [16]:
def user_predict(job_id:str, chat_history:List[Dict[str,str]])->str:
    """
    Receives the name of the user job_id and queues the job into Redis.
    Will loop until getting the answer from our retriver-generative service.

    Parameters
    ----------
    job_id: str
        id of the task
    messages : list[BaseMessage]
        chat_history

    Returns
    -------
    answer:  str
        Model generate an answer as a string 
    """
    
    # Assign an unique ID for this job and add it to the queue.
    # We need to assing this ID because we must be able to keep track
    # of this particular job across all the services
    job_id = str(uuid4())

    # Create a dict with the job data we will send through Redis having the   
    Out_dict = {
        "id": job_id,
        "messages":chat_history,
    }
    job_data = json.dumps(Out_dict)

    # Send the job to the model service using Redis 
    db.lpush(settings.REDIS_QUEUE, job_data)

    # Loop until we received the response from the retriver-generative model
    while True:
        # Attempt to get model predictions using job_id        
        output = db.get(job_id)

        # Check if the text was correctly processed by our model
        if output is not None:
            answer = json.loads(output)['answer']
            
            db.delete(job_id)
            break

        # Sleep some time waiting for model results
        time.sleep(settings.SERVER_SLEEP)

    return answer

In [20]:
# simule user interaction
def run_chatbox():
    chat_history = []
    job_id = str(uuid4())
    while True:
        query = input('User: ')
        if query == 'exit': 
            return output, chat_history
        chat_history.append({'role':'user', 'content':query})

        # Run the agent with the query and chat history
        try:
            output = json.loads(user_predict(job_id, chat_history))['action_input']
        except ValueError:
            output=user_predict(job_id, chat_history)

        print(f"Query:{query},\nAnswer: {output}")

        # updating chat history
        chat_history.append({'role':'assistant', 'content':output})
        

output, history = run_chatbox()

Query:What is AWS?,
Answer: AWS is a cloud computing platform provided by Amazon that offers a wide range of services including compute, storage, database, and other services. Some examples of AWS services are DynamoDB, a highly scalable, low latency key-value database, and AWS EC2, which provides scalable computing capacity in the cloud. Other services include AWS S3 for object storage, AWS Lambda for serverless computing, and AWS RDS for managed relational databases.
Query:EXIT,
Answer: {'role': 'assistant', 'content': 'Goodbye!'}


In [13]:
history

[{'role': 'user',
  'content': 'What was the total revenue of Atlassian in fiscal year ended june 30 2021?'},
 {'role': 'assistant',
  'content': 'The total revenue of Atlassian in fiscal year ended June 30, 2021 was $2.1 billion.'},
 {'role': 'user',
  'content': 'What was the operating income of Amazon Web Services (AWS) in year ended 2020?'},
 {'role': 'assistant',
  'content': 'The operating income for Amazon Web Services (AWS) in 2020 was $13.5 billion.'}]